In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K

from keras.models import load_model

import tensorflow as tf
import tarfile
import os
import shutil

## Extraction of data

- Relocation of files
- Unzipping data and unarchiving

In [20]:
zips = os.listdir('./CXR8/images/')
zips = list(filter(lambda x: x.endswith('.tar.gz'), zips))
for file in zips:
    path = os.path.join('./CXR8/images/', file)
    tar = tarfile.open(path, "r:gz")
    tar.extractall('./CXR8/')
    tar.close()

In [22]:
for file in zips:
    path = os.path.join('./CXR8/images/', file)
    os.remove(path)

In [1]:
import pandas as pd

In [33]:
train = open('./CXR8/train_list.txt', 'r')
val = open('./CXR8/val_list.txt', 'r')
test = open('./CXR8/test_list.txt', 'r')

train_list = set(train.read().splitlines())
val_list = set(val.read().splitlines())
test_list = set(test.read().splitlines())

# print("Train, val: ", train_list.intersection(val_list))
# print("Train, test: ", train_list.intersection(test_list))
# print("Val, test: ", val_list.intersection(test_list))

In [7]:
data = pd.read_csv('./CXR8/Data_Entry_2017_v2020.csv', index_col='Image Index')
data.head()

,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
Image Index,,,,,,,,,,
00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143
00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171
00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,0.168


In [32]:
train_csv = open('./CXR8/train_label.csv', 'w')
val_csv = open('./CXR8/val_label.csv', 'w')
test_csv = open('./CXR8/test_label.csv', 'w')

In [9]:
dist_csv_col = [
    'FileName',
    'Atelectasis',
    'Cardiomegaly',
    'Effusion',
    'Infiltration',
    'Mass',
    'Nodule',
    'Pneumonia',
    'Pneumothorax',
    'Consolidation',
    'Edema',
    'Emphysema',
    'Fibrosis',
    'Pleural_Thickening',
    'Hernia'
]

In [34]:
train_csv.write(','.join(dist_csv_col)+'\n')
val_csv.write(','.join(dist_csv_col)+'\n')
test_csv.write(','.join(dist_csv_col)+'\n')
for index, row in data.iterrows():
    if index in train_list:
        labels = row['Finding Labels'].split('|')
        labels = list(map(lambda x: 1 if x in labels else 0, dist_csv_col[1:]))
        train_csv.write(index + ',' + ','.join(map(str, labels))+ '\n')
    elif index in val_list:
        labels = row['Finding Labels'].split('|')
        labels = list(map(lambda x: 1 if x in labels else 0, dist_csv_col[1:]))
        val_csv.write(index + ',' + ','.join(map(str, labels))+ '\n')
    elif index in test_list:
        labels = row['Finding Labels'].split('|')
        labels = list(map(lambda x: 1 if x in labels else 0, dist_csv_col[1:]))
        test_csv.write(index + ',' + ','.join(map(str, labels))+ '\n')

In [35]:
train.close()
val.close()
test.close()
train_csv.close()
val_csv.close()
test_csv.close()

In [1]:
from utils.dataloader import get_generator, categories
from utils.model import get_model
from utils.weights import compute_class_freqs, get_weighted_loss
import numpy as np

In [6]:
class_weights_pos = {i: train_pos_freq[i]  for i in range(len(categories))}
class_weights_neg = {i: train_neg_freq[i]  for i in range(len(categories))}
class_weights_pos
class_weights_neg

{0: 0.9029098014364174,
 1: 0.9800380228136882,
 2: 0.898803865652725,
 3: 0.8418356569497254,
 4: 0.9531051964512041,
 5: 0.9451441698352345,
 6: 0.9900058090409801,
 7: 0.9686707858048162,
 8: 0.9671657160963245,
 9: 0.9847116603295311,
 10: 0.9832594000844952,
 11: 0.985794254330376,
 12: 0.9738989226869454,
 13: 0.9983100971694128}

In [ ]:
temp = { 0: 0.9029098014364174,
 1: 0.9800380228136882,
 2: 0.898803865652725,
 3: 0.8418356569497254,
 4: 0.9531051964512041,
 5: 0.9451441698352345,
 6: 0.9900058090409801,
 7: 0.9686707858048162,
 8: 0.9671657160963245,
 9: 0.9847116603295311,
 10: 0.9832594000844952,
 11: 0.985794254330376,
 12: 0.9738989226869454,
 13: 0.9983100971694128}

In [2]:
BATCH_SIZE = 32

In [3]:
train_generator = get_generator(
        dtype='train', batch_size=32, image_size=(1024, 1024))
valid_generator = get_generator(
        dtype='val', batch_size=1, image_size=(1024, 1024))
test_generator = get_generator(
        dtype='test', batch_size=1, image_size=(1024, 1024))

model = get_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC'])

Found 75731 validated image filenames.
Found 10793 validated image filenames.
Found 25596 validated image filenames.
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [4]:
train_labels = np.zeros((32*len(train_generator), 14), dtype=np.int8)
for i in range(len(train_generator)):
        if i+1 == len(train_generator): continue
        if i%100 == 0: print(i)
        train_labels[i*BATCH_SIZE:(i+1)*BATCH_SIZE,
                     :14] = train_generator[i][1]

train_pos_freq, train_neg_freq = compute_class_freqs(train_labels)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300


In [14]:
pos_weights = train_neg_freq
neg_weights = train_pos_freq

In [7]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator,
    verbose=1,
    class_weight=class_weights_pos,
)

Epoch 1/10


2023-04-08 04:39:15.605456: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


: 

: 

In [4]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Epoch 1/5


2023-04-08 00:22:49.996770: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-08 00:22:50.078663: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x113a4e290
2023-04-08 00:22:50.078695: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x113a4e290
2023-04-08 00:22:50.086123: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x113a4e290
2023-04-08 00:22:50.086148: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:418 : NOT_FOUND: could not find registered platform with id: 0x113a4e290


NotFoundError: Graph execution error:

Detected at node 'StatefulPartitionedCall_2' defined at (most recent call last):
    File "/opt/homebrew/Caskroom/miniconda/base/envs/mura/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/homebrew/Caskroom/miniconda/base/envs/mura/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/opt/homebrew/Caskroom/miniconda/base/envs/mura/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/homebrew/Caskroom/miniconda/base/envs/mura/lib/python3.9/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/opt/homebrew/Caskroom/miniconda/base/envs/mura/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/opt/homebrew/Caskroom/miniconda/base/envs/mura/lib/python3.9/site-packages/tornado

In [2]:
import tensorflow as tf

cifar = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar.load_data()
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=100,)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(x_train, y_train, epochs=5, batch_size=64)

Epoch 1/5
608/782 [======================>.......] - ETA: 25s - loss: 5.1430 - accuracy: 0.0422

KeyboardInterrupt: 